In [1]:
import pandas as pd
import numpy as np
import json
import os
from src.features import generate_features
import tweepy
import datetime
%load_ext autoreload
%autoreload 2

In [2]:
with open('./config/viz_params.json') as f:
    viz_params = json.load(f)
# Cfg variables
path = viz_params['path']
top_k = viz_params['top_k']
top_k_fig_path = viz_params['top_k_fig_path']
user_hist_path = viz_params['user_hist_path']
user_hist_zoom_path = viz_params['user_hist_zoom_path']
good_path = viz_params['good_path']
bad_path = viz_params['bad_path']
good_tags = viz_params['good_tags']
bad_tags = viz_params['bad_tags']

with open('./config/polarity.json') as f:
    polarity_params = json.load(f)  
top_k = polarity_params['top_k']
api_keys = polarity_params['api_keys']
marker_tags = polarity_params['marker_tags']
max_posts = polarity_params['max_posts']
start_date = polarity_params['start_date']
end_date = polarity_params['end_date']
date_pattern = polarity_params['twitter_date_pattern']
max_iter = polarity_params['max_iter']
toi_con = polarity_params['toi_con']
num_retrieve_con = polarity_params['num_retrieve_con']
toi_sci = polarity_params['toi_sci']
con_path = polarity_params['con_path']
sci_path = polarity_params['sci_path']

with open(api_keys) as f:
    keys = json.load(f)
consumer_key = keys['consumer_key']
consumer_secret = keys['consumer_secret']
access_token = keys['access_token']
access_token_secret = keys['access_token_secret']    

# this needs a place to fit in
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [3]:
marker_tags

['WuhanVirus', 'hoax']

In [4]:
jsons = [os.path.join(path, name) for name in sorted(os.listdir(path)) if 'dataset' in name]

# Hashtag Polarity
* b = Baseline ROF (rate of frequency) = num top 200 / num tweets
* m = Marker ROF = num top 200 / num subset tweets (ones containing marker tweets)
* hashtag polarity = (m-b)/b

In [16]:
ht_polarity = generate_features.hashtag_polarity(jsons, top_k, marker_ht = marker_tags)

polarity mode
there are 1044009 tweets and 543 subset tweets


In [17]:
ht_polarity.head()

COVID19        0.068193
coronavirus    0.650984
Covid19        0.180114
Coronavirus    0.456768
covid19       -0.105819
dtype: float64

In [39]:
ht_polarity.to_json('ht_polarity.json')

In [7]:
ht_polarity= pd.read_json('ht_polarity.json', typ='series')

# User Polarity

In [7]:
users =  generate_features.collect_all_users(jsons)

In [8]:
example_user = 'DrMeghanWalls'

In [9]:
norm_pol = generate_features.user_polarity(api, example_user,
                                           set(ht_polarity.index), 
                                           start_date, end_date, date_pattern,
                                           ht_polarity, 
                                           max_posts = max_posts, 
                                           max_iter=max_iter,
                                          )

investigating DrMeghanWalls


In [10]:
norm_pol

-0.48741066089903107

# Inquiry on a single tweet

In [8]:
retweets = api.retweets(toi_con, num_retrieve_con)

In [9]:
up_dict = generate_features.investigate_retweets(api, toi_sci, num_retrieve_con,
                                           set(ht_polarity.index), 
                                           start_date, end_date, date_pattern,
                                           ht_polarity, sci_path,
                                           max_posts = max_posts, 
                                           max_iter=max_iter)

In [13]:
up_dict

NameError: name 'up_dict' is not defined